In [ ]:
import boto3
import pandas as pd
from tabulate import tabulate

In [ ]:
region_name="us-east-1"

In [ ]:
dynamo_client = boto3.client("dynamodb", region_name=region_name)
dynamo_resource = boto3.resource("dynamodb", region_name=region_name)

In [ ]:
reservations_table_name = next(t for t in dynamo_client.list_tables()["TableNames"] if "bedrock-agents-cdk" in t and "restaurantreservations" in t and "v2" in t)

In [ ]:
sql_queries_table_name = next(t for t in dynamo_client.list_tables()["TableNames"] if "bedrock-agents-cdk" in t and "sqlqueries" in t and "v2" in t)

In [ ]:
reservations_table = dynamo_resource.Table(reservations_table_name)
sql_queries_table = dynamo_resource.Table(sql_queries_table_name)

In [ ]:
def truncate_table(table_name):
    table = dynamo_resource.Table(table_name)
    
    #get the table keys
    tableKeyNames = [key.get("AttributeName") for key in table.key_schema]

    #Only retrieve the keys for each item in the table (minimize data transfer)
    projectionExpression = ", ".join('#' + key for key in tableKeyNames)
    expressionAttrNames = {'#'+key: key for key in tableKeyNames}
    
    counter = 0
    page = table.scan(ProjectionExpression=projectionExpression, ExpressionAttributeNames=expressionAttrNames)
    with table.batch_writer() as batch:
        while page["Count"] > 0:
            counter += page["Count"]
            # Delete items in batches
            for itemKeys in page["Items"]:
                batch.delete_item(Key=itemKeys)
            # Fetch the next page
            if 'LastEvaluatedKey' in page:
                page = table.scan(
                    ProjectionExpression=projectionExpression, ExpressionAttributeNames=expressionAttrNames,
                    ExclusiveStartKey=page['LastEvaluatedKey'])
            else:
                break
    print(f"Deleted {counter}")

In [ ]:
df = pd.DataFrame(reservations_table.scan()["Items"])
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
df = pd.DataFrame(sql_queries_table.scan()["Items"])
if df.shape[0]:
  df = df.sort_values("timestamp_utc")
print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# truncate_table(reservations_table_name)
# truncate_table(sql_queries_table_name)